# Deep Learning - Autoencoder

In [1]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.layers import Input, Dense
from keras.models import Model
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
# Set parameters
image_length = 784
encoding_dim = 200
encoding_dim_2 = 45

train_size = 5000
validation_size = 5000

In [3]:
# Import and convert data to percentage points (divide by 255, the max gray scale value)
x_train = pd.read_csv("data/mnist_train.csv", header = None)
y_train = x_train.loc[:, 0].to_numpy()
x_train = x_train.loc[:, 1:].to_numpy()
x_train = x_train.astype('float32') / 255.

x_test = pd.read_csv("data/mnist_test.csv", header = None)
y_test = x_test.loc[:, 0].to_numpy()
x_test = x_test.loc[:, 1:].to_numpy()
x_test = x_test.astype('float32') / 255.

# Take samples for training and testing. Note that we cannot take the test set yet, as we will use this when the
# full network is completed in R
x_validation = x_train[train_size:(train_size+validation_size), :]
x_train_input = x_train[(train_size+validation_size):, :]
x_train = x_train[:train_size, :]
y_train_input = y_train[(train_size+validation_size):]

# Create one-hot-encoded versions of the data
y_train_input_ohe = to_categorical(y_train_input)
y_test_ohe = to_categorical(y_test)

In [4]:
# Create autoencoder with topology 784 -> 200 -> 45
input_img = Input(shape = (image_length,))
encoded = Dense(encoding_dim, activation='relu')(input_img)
encoded = Dense(encoding_dim_2, activation='relu')(encoded)

decoded = Dense(encoding_dim, activation='relu')(encoded)
decoded = Dense(image_length, activation='sigmoid')(decoded)

encoder = Model(input_img, encoded)

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
# Grid Search - after finding epochs=12 and batch_size=128 was the best
best_acc = 0

for epoch in range(12, 20):
    print(f"Epochs: {epoch}")
    for batch_size in [64, 128]:
        print(f"Batch size: {batch_size}")
        
        autoencoder = Model(input_img, decoded)
        autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
        history = autoencoder.fit(x_train, x_train,
                          epochs=epoch, batch_size=batch_size,
                          shuffle=True, verbose = 0,
                          validation_data=(x_validation, x_validation))
        
        if (history.history['loss'][-1] > history.history['val_loss'][-1]):
            print("Overfitting!")
            next
            
        validation_acc = history.history['val_acc'][-1]
        
        if (validation_acc > best_acc):
            best_acc = validation_acc
            best_epoch = epoch
            best_batch_size = batch_size
            
print(f"Best accuracy: {best_acc}, best epochs: {best_epoch}, best batch_size: {best_batch_size}")

Epochs: 12
Batch size: 64
Instructions for updating:
Use tf.cast instead.
Batch size: 128
Epochs: 13
Batch size: 64
Batch size: 128
Epochs: 14
Batch size: 64


In [ ]:
# Use best parameters found above
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
history = autoencoder.fit(x_train, x_train,
                          epochs=best_epoch, batch_size=best_batch_size,
                          shuffle=True, verbose = 2,
                          validation_data=(x_validation, x_validation))

In [ ]:
# Plot images
encoded_images = encoder.predict(x_train_input)
decoded_images = autoencoder.predict(x_train_input)

plt.figure(figsize=(40, 4))
for i in range(10):
    # Display original images
    ax = plt.subplot(3, 20, i + 1)
    plt.imshow(x_train_input[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # Display encoded images
    ax = plt.subplot(3, 20, i + 1 + 20)
    plt.imshow(encoded_images[i].reshape(9,5))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # Display decoded images
    ax = plt.subplot(3, 20, 2*20 +i+ 1)
    plt.imshow(decoded_images[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
plt.show()

In [ ]:
# Now that the autoencoding part is done, we add the ANN part from exercise 4
full_model = Model(autoencoder.input, autoencoder.layers[-3].output)
full_model.summary()

In [ ]:
H = 20 # H = 20 from exercise 4

layer_1 = Dense(H, activation='relu')(full_model.layers[-1].output)
layer_2 = Dense(10, activation = 'softmax')(layer)
full_model = Model(input_img, layer_2)
full_model.summary()
full_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = full_model.fit(x_train_input, y_train_input_ohe,
                         epochs=epoch, batch_size=batch_size,
                         shuffle=True, verbose=2,
                         validation_data=(x_test, y_test_ohe))

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()